# Load(url), save to file

In [ ]:
# import urllib.request
# url = 'http://cirtec.ranepa.ru/Word2Vec/fixes.stem.txt'
# response = urllib.request.urlopen(url)
# data = response.read()      # a `bytes` object

In [ ]:
# with open('../initial_data/Word2Vec__fixes.stem.txt', 'wb') as f:
#     f.write(data)

# load file

In [ ]:
%%time
with open('../initial_data/Word2Vec__fixes.stem.txt', 'rb') as f:
    data = f.read()
text = data.decode('utf-8') # a `str`; this step can't be used if data is binary
del data
print("len_text: {}".format(len(text)))
sentences = [sentence.lower() for sentence in text.split("\n") if len(sentence) > 60]
del text
print("len_sentences: {}".format(len(sentences)))
print(sentences[0])

In [ ]:
import pymorphy2

from base_defs import get_average_len_words, get_n_gram, drop_all_delimiters, get_gram_vocab
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize

morph = pymorphy2.MorphAnalyzer()

stop_words = ["{}_{}".format(word, morph.parse(word)[0].tag.POS).lower() for word in stopwords.words('russian')]

In [ ]:
%%time
from base_defs import get_average_len_words, get_n_gram, drop_all_delimiters, get_gram_vocab

min_average_len_words = 9
sentences = [[word for word in sentence.split() if word not in stop_words] for sentence in sentences
             if get_average_len_words(sentence) >= min_average_len_words]
print(len(sentences))
print(sentences[0][:5])

In [ ]:
%%time
# # part
# grams = get_n_gram(sentences=sentences[:10000], n=4, delimiters=(b"@", b"#", b"$", b"%"))

# all
grams = get_n_gram(sentences=sentences, n=4, delimiters=(b"@", b"#", b"$", b"%"))

In [ ]:
del sentences

In [ ]:
%%time
import pandas as pd

grams_vocab = []
for ind in range(2, 5):
     grams_vocab.extend(get_gram_vocab(grams=grams, n=ind, delimiters=("@", "#", "$", "%")))
res = pd.DataFrame(grams_vocab, columns=["text", "freq"]).drop_duplicates(["text"]).sort_values(["freq"], ascending=False).reset_index(drop=True)
res["len_text"] = res.text.apply(lambda x: len(x.split()))
res['text_without_tags'] = res['text'].apply(lambda x: " ".join([word.split("_")[0] for word in x.split()]))

In [ ]:
res.head(3)

In [4]:
def save_csv(res_df, path, len_text, head_size=3000):
    save = res_df[res_df.len_text == len_text][["text_without_tags", "freq"]]
    size = min((head_size, save.shape[0]))
    print("All len: {}, save: {}".format(save.shape[0], size))
    
    filename = "{}/frequency_analysis_stem_without_stopwords_{}__{}.csv".format(path, len_text, size)
    print(filename)
    save.head(head_size).to_csv(filename, index=False)

In [ ]:
save_csv(res, path="../resulting_data", len_text=2)
save_csv(res, path="../resulting_data", len_text=3)
save_csv(res, path="../resulting_data", len_text=4)